In [1]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn import preprocessing
import numpy_indexed as npi
from scipy.stats import skew

from scipy.stats import kurtosis

In [148]:
PATHSAMPLES = '../data/training_list.txt'
NSAMPLES= 1000
SOUNDS =np.array(['yes','no','right','five','nine'])


In [149]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    

    
    return matrixMajor

In [150]:
def build_features(n_mfcc,NSAMPLES, PATHSAMPLES, SOUNDS, n_groups=10):
    N_rows = NSAMPLES*len(SOUNDS)*n_mfcc
    N_colums = n_mfcc +3# número de columnas del mfcc + 3 columnas (la palabra y el speaker id, path al que pertenece)
    # el path es necesario ya que estamos teniendo un enfoque de multiples instancias.
    path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

    matrix_features = np.empty((1,n_groups*n_mfcc+2)) # inicialmente

    #Encode etiqueta del audio,speaker y la palabra
    lePath = preprocessing.LabelEncoder()
    leSpeaker = preprocessing.LabelEncoder()
    leWord =  preprocessing.LabelEncoder()
    
    lePath.fit(path_word_speakerId[:,0])
    leSpeaker.fit(path_word_speakerId[:,2])
    leWord.fit(path_word_speakerId[:,1])
    
        
    for sound_info in path_word_speakerId:
        audio_path = '../'+sound_info[0]
        x , sr = librosa.load(audio_path)
        x_normalize=sk.preprocessing.minmax_scale(x, axis=0)       
        mfccs = librosa.feature.mfcc(x_normalize, sr=sr,n_mfcc=n_mfcc,hop_length=int(0.010*sr), n_fft=int(0.025*sr))
        mfccs = mfccs.T
        
        sliceM = mfccs.shape[0]//n_groups
          
        split_mean = np.array([np.mean(arr, axis=0) for arr in np.array(np.array_split(mfccs, n_groups))])
        split_mean = np.reshape(split_mean, (1, split_mean.flatten().shape[0]))

        
        speaker_id = sound_info[2]
        word=sound_info[1]        
        
        labelSpeaker = leSpeaker.transform([speaker_id])
        labelWord= leWord.transform([word])   
           
        fila_P_W_S = np.array([labelWord,labelSpeaker])  
        fila_P_W_S =fila_P_W_S.T
        
        mfccFull =  np.hstack((split_mean,fila_P_W_S))   
    
        matrix_features = np.concatenate((matrix_features,mfccFull), axis=0)   
        
    return matrix_features[1:,:]

In [151]:
matrixFinal = build_features(20,NSAMPLES, PATHSAMPLES, SOUNDS)
print(matrixFinal.shape)

(5000, 202)


In [152]:
np.savetxt('../data/audios_MFCC_Flatten_Means.csv',matrixFinal,delimiter=",")

In [153]:
data_avg = pd.read_csv('../data/audios_MFCC_Flatten_Means.csv', sep=",", header=None)
data_avg = data_avg.values #Convertir en numpy array
data_avg.shape

(5000, 202)